# Sentiment classification - close to the state of the art

Submitted By: Shaikh Muhammad Asjad Raza







Note: I have used 50% train and 50% test split. I did not change spliting criteria. 

I recieved the announcement form professor regarding updated splitting thresholds for train/ valid/ test but i had already trained the models at that time.

This Notebook contains:
1. Data Preprocessing
2. LSTM with trainable Keras Embedding
3. BERT without Ktrain 
4. BERT with Ktrain

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries for preprocessing
import numpy as np
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'ner'])
import re

# Data download

In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzf aclImdb_v1.tar.gz
!ls

--2020-12-04 10:15:01--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  52.5MB/s    in 1.5s    

2020-12-04 10:15:02 (52.5 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]

aclImdb  aclImdb_v1.tar.gz  drive  sample_data


# Alternative with tf.datasets

In [ ]:
!pip install tensorflow-datasets > /dev/null

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
(ds_train,ds_test),ds_info = tfds.load(
    name="imdb_reviews",
    split=["train","test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2UZNEV/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2UZNEV/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2UZNEV/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
ds_info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [ ]:
# print and see a few examples to get familiar with the data
train_examples_batch, train_labels_batch = next(iter(ds_train.batch(20)))
train_examples_batch, train_labels_batch

(<tf.Tensor: shape=(20,), dtype=string, numpy=
 array([b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.',
        b'Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the la

In [ ]:
# converting raw data into DataFrame as i am more comfortable with
# Pandas DataFrames

df_train = pd.DataFrame()
df_valid = pd.DataFrame()

for i in ds_train.as_numpy_iterator():
      new_row = {'reviews':i[0], 'label':int(i[1])}
      df_train = df_train.append(new_row, ignore_index=True)
df_train['reviews'] = df_train['reviews'].astype(str).str[1:]

for i in ds_test.as_numpy_iterator():
      new_row = {'reviews':i[0], 'label':int(i[1])}
      df_valid = df_valid.append(new_row, ignore_index=True)
df_valid['reviews'] = df_valid['reviews'].astype(str).str[1:]


In [ ]:
# save for future use
df_train.to_pickle('/content/drive/MyDrive/data/df_train')
df_valid.to_pickle('/content/drive/MyDrive/data/df_valid')

# **Load saved train and valid dataframes**

In [ ]:
# loading data
df_train = pd.read_pickle('/content/drive/MyDrive/data/df_train')
df_valid = pd.read_pickle('/content/drive/MyDrive/data/df_valid')

In [ ]:
print(df_train.shape, df_valid.shape)

(25000, 2) (25000, 2)


# **Data Cleaning**

First Concatinate train and valid dataframes and then: 
1. Lower casing
2. Remove Backslash
3. Removal of URLs
4. Removal of HTML tags
5. Removal of Punctuations 
6. Removal of Stopwords
7. Lemmatization
8. Remove digits
6. Spelling Correction
7. Remove repeating words



#### **Concatination**

In [ ]:
# Concatinate train and valid dataframes
# Data cleaning is to be done on both train and valid dataframes
data = pd.concat([df_train, df_valid])
data.shape

(50000, 2)

#### **Lower Casing**

Lower casing so that two words "Like" and "like" are not treated differently.

In [ ]:
data["reviews_lowercased"] = data["reviews"].str.lower()
data.head()

,label,reviews,reviews_lowercased
0,0.0,"""This was an absolutely terrible movie. Don't ...","""this was an absolutely terrible movie. don't ..."
1,0.0,'I have been known to fall asleep during films...,'i have been known to fall asleep during films...
2,0.0,'Mann photographs the Alberta Rocky Mountains ...,'mann photographs the alberta rocky mountains ...
3,1.0,'This is the kind of film for a snowy Sunday a...,'this is the kind of film for a snowy sunday a...
4,1.0,"'As others have mentioned, all the women that ...","'as others have mentioned, all the women that ..."


In [ ]:
data.drop('reviews', inplace=True, axis=1)
data.rename(columns={"reviews_lowercased": "reviews"}, inplace=True)
data.head()

,label,reviews
0,0.0,"""this was an absolutely terrible movie. don't ..."
1,0.0,'i have been known to fall asleep during films...
2,0.0,'mann photographs the alberta rocky mountains ...
3,1.0,'this is the kind of film for a snowy sunday a...
4,1.0,"'as others have mentioned, all the women that ..."


#### **Remove backslash**

I saw many backslashes in the data which won't help in prediction so i will remove them also. 

In [ ]:
def remove_backslash(text):
  r = [i.replace('\\', "") for i in text] 
  return (''.join(r))

data["reviews_w/o_backslash"] = data["reviews"].apply(remove_backslash)

In [ ]:
data.head()

,label,reviews,reviews_w/o_backslash
0,0.0,"""this was an absolutely terrible movie. don't ...","""this was an absolutely terrible movie. don't ..."
1,0.0,'i have been known to fall asleep during films...,'i have been known to fall asleep during films...
2,0.0,'mann photographs the alberta rocky mountains ...,'mann photographs the alberta rocky mountains ...
3,1.0,'this is the kind of film for a snowy sunday a...,'this is the kind of film for a snowy sunday a...
4,1.0,"'as others have mentioned, all the women that ...","'as others have mentioned, all the women that ..."


In [ ]:
data.drop('reviews', inplace=True, axis=1)
data.rename(columns={"reviews_w/o_backslash": "reviews"}, inplace=True)

#### **Removal of URLs**

There were a few URLs in the data that caught my eye so i am avoiding URLS as they won't help in prediction.

In [ ]:
def remove_url(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

data["reviews_w/o_URL"] = data["reviews"].apply(remove_url)

In [ ]:
data.head()

,label,reviews,reviews_w/o_URL
0,0.0,"""this was an absolutely terrible movie. don't ...","""this was an absolutely terrible movie. don't ..."
1,0.0,'i have been known to fall asleep during films...,'i have been known to fall asleep during films...
2,0.0,'mann photographs the alberta rocky mountains ...,'mann photographs the alberta rocky mountains ...
3,1.0,'this is the kind of film for a snowy sunday a...,'this is the kind of film for a snowy sunday a...
4,1.0,"'as others have mentioned, all the women that ...","'as others have mentioned, all the women that ..."


In [ ]:
data.drop('reviews', inplace=True, axis=1)
data.rename(columns={"reviews_w/o_URL": "reviews"}, inplace=True)

#### **Remove HTML tags**

There were many HTML tags in the data. I will remove them also as they are useless for predictions.  

In [ ]:
data['reviews'] = data['reviews'].replace({'<.*?>': ''}, regex = True, inplace=False)

In [ ]:
data['reviews'].iloc[200]

'"this movie was bad to say the least!!! the plethora of superb cars are disgraced to have have been showcased in this lame movie. it starts off with a race from l.a. to las vegas to be completed in 1hr 45min...in a ferrari f430. i did that in 1hr 50min in a tiny 4cylinder 140hp 1993 honda accord. seriously...this movie does not do justice to these cars. obviously these writers are just that and probably drive under the speed limit with their hands @ 10 & 2 o\'clock. i remember seeing on the news how eddie griffin crashed a 1.5 million dollar ferrari enzo going 30-40mph>>> youtube.com/watch?v=cnvrmzx2kms and...the director andy cheng is the biggest sellout!!! he brings shame to his own race. i wonder just how many people he orally pleased just to break into hollywood. he partook in a movie that portrays asian americans in such a negative and false way. asian women>>cheap money grubbing whores. asian men>>losing compulsive gamblers & thugs that get beat all the time . what the heck is a

#### **Removal of Puntuations**

Full stops, commas etc. are being removed.

In [ ]:
# prevent punctuations in words like "doesn't" not to be removed
# They are important for predictions
nlp.tokenizer.rules = {key: value for key, value in nlp.tokenizer.rules.items() if "'" not in key and "’" not in key and "‘" not in key}

In [ ]:
# Using Spacy for this task
def remove_punct(text):
  doc = nlp(text)
  token_list = [token.text for token in doc if token.is_punct != True and token.text != "'s"]
  joint_text = ' '.join(token_list)
  final_removal = "".join(u for u in joint_text if u not in ("?", ".", ";", ":", "!", "-", '"', ')', '(', '+'))
  return (final_removal)

data["reviews_w/o_punct"] = data["reviews"].apply(remove_punct)

In [ ]:
data.head()

,label,reviews,reviews_w/o_punct
0,0.0,"""this was an absolutely terrible movie. don't ...",this was an absolutely terrible movie don't be...
1,0.0,'i have been known to fall asleep during films...,i have been known to fall asleep during films ...
2,0.0,'mann photographs the alberta rocky mountains ...,mann photographs the alberta rocky mountains i...
3,1.0,'this is the kind of film for a snowy sunday a...,this is the kind of film for a snowy sunday af...
4,1.0,"'as others have mentioned, all the women that ...",as others have mentioned all the women that go...


In [ ]:
data.drop('reviews', inplace=True, axis=1)
data.rename(columns={"reviews_w/o_punct": "reviews"}, inplace=True)

In [ ]:
print(data['reviews'].iloc[6])

okay you have penelope keith as miss herringbone tweed bbe backbone of england she killed off in the first scene that right folks this show has no backbonepeter o'toole as ol colonel cricket from the first war and now the emblazered lord of the manorjoanna lumley as the ensweatered lady of the manor 20 years younger than the colonel and 20 years past her own prime but still glamourous brit spelling not mine enough to have a toy boy on the side it alright they have col cricket full knowledge and consent they guy even comes round for christmas still she considerate of the colonel enough to have said toy boy her own age what a galdavid mccallum as said toy boy equally as pointlessly glamourous as his squeeze pilcher couldn't come up with any cover for him within the story so she gave him a hush hush job at the circusand finally susan hampshire as miss polonia teacups venerable headmistress of the venerable girls boarding school serving tea in her office with a dash of deep poignant advice

#### **Removal of stopwords**

Stopwords have to be removed as they create noise in the data.

In [ ]:
# Using Spacy for this task
def remove_stopwords(text):
  doc = nlp(text)
  token_list = [token.text for token in doc if token.is_stop != True]
  return (' '.join(token_list))

data["reviews_w/o_stopwords"] = data["reviews"].apply(remove_stopwords)

In [ ]:
data.head(5)

,label,reviews,reviews_w/o_stopwords
0,0.0,this was an absolutely terrible movie don't be...,absolutely terrible movie don't lured christop...
1,0.0,i have been known to fall asleep during films ...,known fall asleep films usually combination th...
2,0.0,mann photographs the alberta rocky mountains i...,mann photographs alberta rocky mountains super...
3,1.0,this is the kind of film for a snowy sunday af...,kind film snowy sunday afternoon rest world ah...
4,1.0,as others have mentioned all the women that go...,mentioned women nude film absolutely gorgeous ...


In [ ]:
data.drop('reviews', inplace=True, axis=1)
data.rename(columns={"reviews_w/o_stopwords": "reviews"}, inplace=True)

In [ ]:
print(data['reviews'].iloc[1])

known fall asleep films usually combination things including tired warm comfortable sette having eaten lot occasion fell asleep film rubbish plot development constant constantly slow boring things happen explanation causing admit missed film watched majority happen accord real concern nt recommend film


#### **Lemmatization**

Doing this so that words like "running" and "run" and not treated differently. 
This will convert all the words to their lemmas. 

In [ ]:
# Using Spacy for this task
def lemmatize(text):
  doc = nlp(text)
  token_list = [token.lemma_ for token in doc]
  return (' '.join(token_list))

data["reviews_lemmatized"] = data["reviews"].apply(lemmatize)

In [ ]:
data.head(5)

,label,reviews,reviews_lemmatized
0,0.0,absolutely terrible movie don't lured christop...,absolutely terrible movie don't lure christoph...
1,0.0,known fall asleep films usually combination th...,know fall asleep film usually combination thin...
2,0.0,mann photographs alberta rocky mountains super...,mann photograph alberta rocky mountain superb ...
3,1.0,kind film snowy sunday afternoon rest world ah...,kind film snowy sunday afternoon rest world ah...
4,1.0,mentioned women nude film absolutely gorgeous ...,mention woman nude film absolutely gorgeous pl...


In [ ]:
data.drop('reviews', inplace=True, axis=1)
data.rename(columns={"reviews_lemmatized": "reviews"}, inplace=True)

In [ ]:
data['reviews'].iloc[2000]

"love showevery time watch episode repeat line remind good huge sci fi fan ground important science fiction history film tv theory universe start religion amaze season season get well betteri've fan macgyver 5 19 find ironic 2 favorite tv show time star richard dean anderson interest character practically opposite otherback see stargate movie instantly like consider favorite sci fi flick hear tv spin get excite didn't showtime till 5 season overthough i'm disappoint hear roland emmerich dean devlin want trilogy movie studio option series instead have turn fine maybe betterthis amaze hope die atlantis come"

#### **Remove Digits**

In my opinion having digits in reviews might not help in prediction. 

In [ ]:
# Using spacy for this task
def remove_digits(text):
  doc = nlp(text)
  token_list = [token.text for token in doc if token.is_digit != True]
  return (' '.join(token_list))

data["reviews_w/o_digits"] = data["reviews"].apply(remove_digits)

In [ ]:
data.drop('reviews', inplace=True, axis=1)
data.rename(columns={"reviews_w/o_digits": "reviews"}, inplace=True)

In [ ]:
print(data['reviews'].iloc[102])

dutch language film paul verhoeven go mainstream hollywood film basic instinct robocop total recall set stage open story black widow spider catch prey web meet gerard reve annoy self center writer morbid imagination gerard invite guest speaker literary club meet sea town hour amsterdam verhoeven let glimpse gerard imagination twist reality ask writer bite close insanity admit read newspaper say boom read doom say flood read blood say red dead tell story time begin believe lie truth accept offer overnight guest club treasurer beautiful wealthy salon owner get know learn husband die begin imagine black widow reality twist murderess psychological drama recount old film stick memory figure favorite gender look body work paul verhoeven master make question understand reality nice change pace usual hollywood fare see film stick


#### **Spelling correction**

Correct any spelling mistakes so that same words with wrong spellings are not treated differently.

In [ ]:
#!pip install pyspellchecker

In [ ]:
# using spellchecker library for this task
from spellchecker import SpellChecker

spell = SpellChecker()

# Don't correct the following words
spell.word_frequency.load_words(["doesn't", "isn't", "won't"])

def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
          pass
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
data["reviews_w_correct_spellings"] = data["reviews"].apply(correct_spellings)

In [ ]:
data.drop('reviews', inplace=True, axis=1)
data.rename(columns={"reviews_w_correct_spellings": "reviews"}, inplace=True)

#### **Remove repeating words**

Since i have cleaning most of the data as of now, there are many cases where same words are repeating right after one another. eg. "Asjad **likes like**s to **eat eat** apple". 

**Note:** This won't remove repeating words that are repeated at two diffierent locations in the review.

In [ ]:
# using regex for this purpose
def remove_repetition(text):
  re_output = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)
  return re_output

data["reviews_w/o_repetition"] = data["reviews"].apply(remove_repetition)

In [ ]:
data.drop('reviews', inplace=True, axis=1)
data.rename(columns={"reviews_w/o_repetition": "reviews"}, inplace=True)

#### **Compare the preprocessed and raw version of the data**

In [ ]:
# Cleaned data
for i in range(10):
  print(data['reviews'].iloc[i])

absolutely terrible movie don't lure christopher walken michael ironside great actor simply wrong role history great act redeem movie ridiculous storyline movie early ninety propaganda piece pathetic scene columbian rebel make case revolution mare conchita alonso appear phony pseudo love affair walken pathetic emotional plug movie devoid real mean disappoint movie like ruin actor like christopher walken good barely sit
know fall asleep film usually combination thing include tire warm comfortable have eat lot occasion fall asleep film rubbish plot development constant constantly slow bore thing happen explanation cause admit miss film watch majority happen accord real concern not recommend film
mann photograph alberta rocky mountain superb fashion jimmy stewart walter brennan enjoyable performance come hollywood mountie tell people dawson city yukon elect marshal yes marshal enforce law battle street control town remotely resemble happen canadian border klondike gold rush mr mann compan

In [ ]:
# Raw data
for i in range(10):
  print(df_train['reviews'][i])

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was c

#### **Divide the data back to train and valid**

In [ ]:
# pickel the data so that i don't have to do the preprosessing again.
data.to_pickle('/content/drive/MyDrive/data/data')

In [ ]:
# read the saved data
data = pd.read_pickle('/content/drive/MyDrive/data/data')

From now onwards i'll call **'test data'** as **'valid data'** because i'll use test data as for validation purpose.

In [ ]:
# Split the data back to 50% train and 50% valid
train_df = data.iloc[0:25000, 1]
y_train = data.iloc[0:25000, 0]
valid_df = data.iloc[25000:, 1]
y_valid = data.iloc[25000:, 0]
print(train_df.shape, valid_df.shape)

(25000,) (25000,)


# **Models**

###**Tokenize for LSTM**

In [ ]:
# find the average review length
# just to get an idea of where to sent max_sent_length
review_length = []
def get_length(text):
  rev_length = len(text.split())
  review_length.append(rev_length)
  return review_length

length_of_sentiments= data["reviews"].apply(get_length)
avg_seq_length = int(np.ceil(sum(length_of_sentiments.iloc[0])/len(length_of_sentiments.iloc[0])))
print("Average sentiment length: " + str(avg_seq_length))

Average sentiment length: 98


In [ ]:
max_seq_length = 150
# size of word index
vocab_size = 31594

In [ ]:
# Import Keras Tokenizer and sequence
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

In [ ]:
# This function will tokenize the data using Keras Tokenizer
# Padding will also be done so that all reviews are of same length

def tokenize_padding_seq(train_df, valid_df, max_features, max_seq_length):
   
    tokenizer = Tokenizer(num_words=max_features, split=' ', lower=True)

    tokenizer.fit_on_texts(train_df)
    X_train = tokenizer.texts_to_sequences(train_df)
    X_valid  = tokenizer.texts_to_sequences(valid_df)

    X_train = sequence.pad_sequences(X_train, maxlen=max_seq_length, padding='post', truncating='post')
    X_valid  = sequence.pad_sequences(X_valid,  maxlen=max_seq_length, padding='post', truncating='post')
    
    # Tokeneizer gives more tokens then that of the actual vocab size
    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))
    return X_train, y_train, X_valid, y_valid

In [ ]:
X_train, y_train, X_valid, y_valid = tokenize_padding_seq(train_df, valid_df, max_features=vocab_size, max_seq_length=max_seq_length)

Found 31594 unique tokens.


In [ ]:
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)

(25000, 150) (25000,) (25000, 150) (25000,)


### **LSTM with Keras Embedding**

In [ ]:
# Import the necessary libraries
from tensorflow.keras.layers import LSTM, Input, Embedding, Dense, Dropout, SpatialDropout1D, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as be
from tensorflow.keras.optimizers import Adadelta, Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.initializers import glorot_normal
from tensorflow.keras.models import load_model

import tensorflow as tf

In [ ]:
# Define the embedding size
embedding_size = 200
LSTM_size = 64
dropout_rate = 0.2

#### **Model**

In [ ]:
# reset the graph / clear session
be.clear_session()
tf.compat.v1.reset_default_graph()


# Input layer 
inputs = Input(shape = (max_seq_length, ))

# Embedding layer
embedding_layer = Embedding(vocab_size+1, embedding_size, input_length=max_seq_length, mask_zero=True)(inputs)

# LSTM layer 1
LSTM_1 = LSTM(LSTM_size, kernel_initializer=glorot_normal(seed=19), return_sequences = False)(embedding_layer)

# Dropout
Dropout_1 = Dropout(dropout_rate, seed=19)(LSTM_1)

# Output Layer
# It's binary classification, one unit with sigmoid activation
predictions = Dense(1, activation = 'sigmoid')(Dropout_1)

# Instantiate the model
model = Model(inputs,predictions)

# get the model summary
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 200)          6318800   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                67840     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 6,386,705
Trainable params: 6,386,705
Non-trainable params: 0
_________________________________________________________________


#### **Loss and optimization**

In [ ]:
# Loss 
loss = 'binary_crossentropy'

# Optimizer
optimizer = Adam(learning_rate=0.0001) 
 
# Compilation
model.compile(loss=loss, optimizer= optimizer, metrics = ['accuracy'])

#### **Training**

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/checkpoint_lstm/temp'

# Save the weights based on min validation loss. 
# even if model overfits i'll use this checkpoint to get the weights which had min valid loss.
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_loss', mode='min', save_best_only=True)

# I will use the entire train data for training and use the test data (which i renamed 
# as valid data) for validation.

# Loads the weights (IF NEEDED)
#model.load_weights(checkpoint_filepath)
with tf.device('/gpu:0' and '/gpu:1'):
  history = model.fit(X_train, 
                      y_train,
                      epochs=5,
                      validation_data=(X_valid, y_valid),
                      batch_size=128,
                      verbose=1,
                      shuffle=False,
                      callbacks=[model_checkpoint_callback])

Epoch 1/5
196/196 [==============================] - 102s 523ms/step - loss: 0.6233 - accuracy: 0.6826 - val_loss: 0.4673 - val_accuracy: 0.8203
Epoch 2/5
196/196 [==============================] - 102s 520ms/step - loss: 0.3492 - accuracy: 0.8694 - val_loss: 0.3548 - val_accuracy: 0.8520
Epoch 3/5
196/196 [==============================] - 107s 547ms/step - loss: 0.2427 - accuracy: 0.9135 - val_loss: 0.3682 - val_accuracy: 0.8503
Epoch 4/5
196/196 [==============================] - 102s 521ms/step - loss: 0.1858 - accuracy: 0.9388 - val_loss: 0.3938 - val_accuracy: 0.8452
Epoch 5/5
196/196 [==============================] - 102s 520ms/step - loss: 0.1488 - accuracy: 0.9550 - val_loss: 0.3889 - val_accuracy: 0.8347


In [ ]:
# get the best weights and evaluate on valid set
model.load_weights(checkpoint_filepath)
scores = model.evaluate(X_valid, y_valid, verbose=1)

782/782 [==============================] - 25s 32ms/step - loss: 0.3548 - accuracy: 0.8520


Observations:
1. Model tends to overfit despite the use of dropout.
2. In my opinion, given the data preprocessing that i have done, the Model has reached the generalization capasity.
3. Getting more training data may help in this case (Since I have used 50% train and 50% Valid).

###  **BERT without Ktrain Wrapper (Using Keras and Tensorflow)**

Note: The following code for BERT is generted by watching multiple tutorials on YouTube because I had no clue of how to 'implement' BERT without Ktrain. The brighter side of the story is that now i have an idea about how to implement BERT without wrapper. My understanding of the implementation can be judged by the comments i have made side by side. 

I will train the entire model and won't do the fine tuning. 

In [ ]:
# istall the latest version of tensorflow GPU
!pip install tensorflow-gpu >> /dev/null

In [ ]:
# required for using dependencies
!pip install --upgrade grpcio >> /dev/nul l

In [ ]:
# istall tqdm for the progress bars
!pip install tqdm >> /dev/null

In [ ]:
# requried because the original implementation for bert is not campatble with tensorflow 2
# this will allow to use pretrained BERT model with tf 2
!pip install bert-for-tf2 >> /dev/null

In [ ]:
!pip install sentencepiece >> /dev/null

In [ ]:
# import dependencies 
import os
import math 
import datetime

from tqdm import tqdm

import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
# Bert also comes with its own tokenizer, i will use this for pre-processing
from bert.tokenization.bert_tokenization import FullTokenizer

There are 2 pre-trained version of BERT. I will use base BERT with 12 encoders.

In [ ]:
# download the model
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# L-12 tells the number of encoders (12)
# H-786 tell the Hiden size (786)
# just the lower case version
# zip file stored locally on colab

--2020-12-03 10:17:32--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.8.16, 172.253.63.128, 142.250.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.8.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   148MB/s    in 2.6s    

2020-12-03 10:17:35 (148 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [ ]:
# unzip the file
!unzip uncased_L-12_H-768_A-12

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


I get a checkpoint and vocab that will be used by the tokenizer that comes with BERT. It also has bert_config.json which contains configuration for importing the model.



```
# check bert_config.json
{
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}
```



In [ ]:
# make a folder and move everthing that i downloaded to the folder
os.makedirs('model', exist_ok=True)
# move it to the folder
!mv uncased_L-12_H-768_A-12/ model

In [ ]:
# specify the model name
bert_model_name = 'uncased_L-12_H-768_A-12'
# specfy the checkpoint directory
bert_checkpoint_dir = os.path.join('model/', bert_model_name)
# specify the checkpoint file itself
bert_checkpoint_file = os.path.join(bert_checkpoint_dir, 'bert_model.ckpt')
# specify the config file, the ".json" file
bert_config_file = os.path.join(bert_checkpoint_dir, 'bert_config.json')

#### **Preprocessing**

In [ ]:
# get the training and valid data
train_df = data.iloc[0:25000]
valid_df = data.iloc[25000:]
print(train_df.shape, valid_df.shape)

(25000, 2) (25000, 2)


In [ ]:
 class SentimentClassification:
  DATA_COLUMN = "reviews" 
  LABEL_COLUMN = "label"

  # define a constructor
  # type of tokenizer = FullTokenizer
  def __init__(self, train, valid, tokenizer: FullTokenizer, labels, max_seq_len):
    self.tokenizer = tokenizer

    # max_seq_len = 0 because i'll see later on what is the real max_seq_length
    self.max_seq_len = 0
    self.labels = labels
    
    # create X_train, y_train, X_valid, y_valid
    # map the prepare method that will be created later on, to the train and valid data
    #train, valid = map(lambda df: df.reindex(df[SentimentClassification.DATA_COLUMN].str.len().sort_values().index), [train, valid])
    ((self.X_train, self.y_train), (self.X_valid, self.y_valid)) = map(self._prepare, [train, valid])
    
    # see the max_seq_length
    # calculated from both training and validation data
    print("max seq_len", self.max_seq_len)
    self.max_seq_len = min(self.max_seq_len, max_seq_len)

    # do the padding by mapping pad method on train and validation data
    # pad the sequences in case of mismatch with max_seq_length
    self.X_train, self.X_valid = map(self._pad, [self.X_train, self.X_valid])
  
  # define the prepare method
  def _prepare(self, df):
    x, y = [], [] # create two lists for reviews 'x' and labels 'y'

    # iterate over each row in the dataframe, using tqdm to follow the progress
    for _, row in tqdm(df.iterrows()):
      review, label = row[SentimentClassification.DATA_COLUMN],row[SentimentClassification.LABEL_COLUMN] # extract review and label
      tokens = self.tokenizer.tokenize(review) # tokenize the reviews

      # Add two special tokens, 'CLS' and 'SEP'
      # CLS means a sequence is starting
      # SEP means seperator token, defines a seq has ended
      tokens = ["[CLS]"] + tokens + ["[SEP]"]
      token_ids = self.tokenizer.convert_tokens_to_ids(tokens) # convert the tokens into numbers

      # calculate the max_seq_length
      # max of max_seq_len which is previously defined as 0 and length of token_ids
      self.max_seq_len = max(self.max_seq_len, len(token_ids))

      # finally append the token_ids and labels
      x.append(token_ids)
      y.append(self.labels.index(label))

    return np.array(x), np.array(y)

  # define the pad method
  # applied only on token ids that i previosly created
  def _pad(self, ids):
    x = []
    for input_ids in ids:
      
      # I have to igonre the two tokens that i have previosly created 
      # hence here i have max_seq_len - 2
      cut_point = min(len(input_ids), self.max_seq_len - 2)
      input_ids = input_ids[:cut_point]

      # if the max_seq_len = 100 and input_ids = 55 then, it will append (post) 
      # 45 zeros to the sequence 
      input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
      x.append(np.array(input_ids))
    return np.array(x)

In [ ]:
# i have to create FullTokenizer
# i am using the uncased version of BERT therefore dolowercase=True
tokenizer = FullTokenizer(vocab_file = os.path.join(bert_checkpoint_dir, 'vocab.txt'), do_lower_case=True)

In [ ]:
labels = train_df.label.unique().tolist()
data = SentimentClassification(train_df, valid_df, tokenizer, labels, max_seq_len=250)

25000it [00:43, 577.39it/s]
25000it [00:42, 595.11it/s]


max seq_len 1366


#### **Model**

In [ ]:
# accepts the max_seq_len & bert_checkpoint_file
def create_model(max_seq_len, bert_checkpoint_file, bert_config_file):
  
  # open the file
  with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert_params.adapter_size = None # I am not fine tuning bert. I set it to None for now.
      bert = BertModelLayer.from_params(bert_params, name="bert") # huge Keras BERT layer

  # reset the graph
  be.clear_session()
  tf.compat.v1.reset_default_graph()
  
  dropout_rate = 0.5
  ### Model

  # Input shape same as with LSTM
  input = Input(shape=(max_seq_len, ), name= 'input') 
  
  # BERT layer
  bert_output = bert(input) 
  print(bert_output.shape)

  # flatten bert_output
  cls_out = Lambda(lambda seq: seq[:, 0, :])(bert_output)
  dropout_1 = Dropout(dropout_rate)(cls_out)

  # Now i make a small classifier that takes in the secquences from bert
  # number of units is equal to hidden units of bert
  dense_1 = Dense(units=768, activation="relu")(dropout_1)
  dropout_2 = Dropout(dropout_rate)(dense_1)

  # output layer
  predictions = Dense(1, activation="sigmoid")(dropout_2)

  model = Model(inputs=input, outputs=predictions)
  #model.build(input_shape=(None, max_seq_len))
  
  # load the pre_trained weights
  load_stock_weights(bert, bert_checkpoint_file)
      
  return model

In [ ]:
# call the create_model function
model = create_model(data.max_seq_len, bert_checkpoint_file, bert_config_file)
model.summary() 

(None, 250, 768)
Done loading 196 BERT weights from: model/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7fc995609ba8> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 250)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 250, 768)          108

#### **Loss and Optimization**

In [ ]:
# Loss 
loss = 'binary_crossentropy'

# Optimizer
optimizer = Adam(learning_rate=1e-5) 
 
# Compilation
model.compile(loss=loss, optimizer= optimizer, metrics = ['accuracy'])

#### **Training**

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/bert_chk/checkpoint'
# Save the weights based on min validation loss. 
# even if model overfits i'll use this checkpoint to get the weights which had min valid loss.
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_loss', mode='min', save_best_only=True)

# Loads the weights (IF NEEDED)
#model.load_weights(checkpoint_filepath)
history = model.fit(data.X_train, 
                    data.y_train,
                    epochs=3,
                    validation_data=(data.X_valid, data.y_valid),
                    batch_size=6,
                    verbose=1,
                    shuffle=True,
                    callbacks=[model_checkpoint_callback])

Epoch 1/3
4167/4167 [==============================] - 2848s 683ms/step - loss: 0.4625 - accuracy: 0.7565 - val_loss: 0.3020 - val_accuracy: 0.8684
Epoch 2/3
4167/4167 [==============================] - 2885s 692ms/step - loss: 0.2722 - accuracy: 0.8918 - val_loss: 0.2803 - val_accuracy: 0.8826
Epoch 3/3
4167/4167 [==============================] - 2871s 689ms/step - loss: 0.1835 - accuracy: 0.9320 - val_loss: 0.3099 - val_accuracy: 0.8831


Observartions:
1. Difficult to handle overfitting despite the use of dropout.
2. This implementation is not allowing me to use max_seq_length greater than 250. This could be a potential reason for low validation accuracy as generated embedding by BERT may provide poor summary of the sentiments.
3. Better validation accuracy than LSTMs, logictic Regression and FastText.

Next Steps:
1. Diagnose the problem of limited sequence length, overfitting and poor accuracy. 
2. If time allows i will use the wrapper Ktrain for BERT and compare the results with this implementation. Otherwise, that would a part of next step.
3. learn from mistakes.  

# **BERT with Ktrain**

Finally i will now implement BERT using Ktrain. 




By now i am thinking that maybe i went way to far with Data cleaning that's why the Models are not able to generalize properly. To be sure about this i will first train on less preprocessed data and then use the preprocessed data that i used for all the other models (BERT w/o wrapper, LSTMs, Logistic Regression, FastText). 

This time i am only going to do the following Data preprocessing:
1. Lower Casing
2. Removal of URLs
3. Removal of HTML tags
4. Removal of Puntuations

In [ ]:
# less cleaned data
for i in range(10):
  print(data['reviews'].iloc[i])

this was an absolutely terrible movie don't be lured in by christopher walken or michael ironside both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the columbian rebels were making their cases for revolutions maria conchita alonso appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining actor like christopher walken good name i could barely sit through it
i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the sette and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow an

In [ ]:
# install Ktrain
#!pip install Ktrain

In [ ]:
# import necessary Libraries
import ktrain
from ktrain import text as txt

In [ ]:
# Do the splitting
train_df = data.iloc[0:25000, ]
valid_df = data.iloc[25000:, ]
print(train_df.shape, valid_df.shape)

(25000, 2) (25000, 2)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

# convert the labels from float to positive and negative labels as Ktrain expects
train_df['label'] = train_df['label'].map({1.0: 'positive', 0.0: 'negative'})
valid_df['label'] = valid_df['label'].map({1.0: 'positive', 0.0: 'negative'})

#### **Load/preprocess data**

In [ ]:
# trying a with a bigger size of max_len hyperparameter
(x_train, y_train), (x_test, y_test), preproc = txt.texts_from_df(train_df=train_df, 
                                                                  text_column='reviews', 
                                                                  label_columns='label', 
                                                                  val_df=valid_df, 
                                                                  maxlen=500, 
                                                                  ngram_range=1, 
                                                                  preprocess_mode='bert',
                                                                  random_state=None,
                                                                  verbose=1)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


#### **Load Model**

In [ ]:
model = txt.text_classifier('bert', (x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


#### **Saving and reloading**

In [ ]:
# save model and Preprocessor instance after partially training (checkpoints)
# So that i don't have train again
ktrain.get_predictor(model, preproc).save('/content/drive/MyDrive/bertKtrainChk/checkpoint')

# reload Predictor and extract model
#model = ktrain.load_predictor('/content/drive/MyDrive/bertKtrainChk/checkpoint').model

#### **Wrapping**

In [ ]:
# wrap model and data in ktrain.Learner object
learner = ktrain.get_learner(model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=6)

#### **Training**

In [ ]:
# using 1cycle learning rate schedule for 1 epoch
learner.fit_onecycle(2e-5, 1) 



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 3477s 835ms/step - loss: 0.2514 - accuracy: 0.8955 - val_loss: 0.1623 - val_accuracy: 0.9388


After this i will train BERT on Fully preprocessd data that i used for other models (LSTMs, logistic Regression, FastText)

### **Training BERT on Fully preprocessed data**

In [ ]:
# Extreme cleaned data
for i in range(10):
  print(data['reviews'].iloc[i])

absolutely terrible movie don't lure christopher walken michael ironside great actor simply wrong role history great act redeem movie ridiculous storyline movie early ninety propaganda piece pathetic scene columbian rebel make case revolution mare conchita alonso appear phony pseudo love affair walken pathetic emotional plug movie devoid real mean disappoint movie like ruin actor like christopher walken good barely sit
know fall asleep film usually combination thing include tire warm comfortable have eat lot occasion fall asleep film rubbish plot development constant constantly slow bore thing happen explanation cause admit miss film watch majority happen accord real concern not recommend film
mann photograph alberta rocky mountain superb fashion jimmy stewart walter brennan enjoyable performance come hollywood mountie tell people dawson city yukon elect marshal yes marshal enforce law battle street control town remotely resemble happen canadian border klondike gold rush mr mann compan

In [ ]:
(x_train, y_train), (x_test, y_test), preproc = txt.texts_from_df(train_df=train_df, 
                                                                  text_column='reviews', 
                                                                  label_columns='label', 
                                                                  val_df=valid_df, 
                                                                  maxlen=500, 
                                                                  ngram_range=1, 
                                                                  preprocess_mode='bert',
                                                                  random_state=None,
                                                                  verbose=1)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = txt.text_classifier('bert', (x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
# wrap model and data in ktrain.Learner object
learner = ktrain.get_learner(model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=6)

In [ ]:
# using 1cycle learning rate schedule for 1 epoch
learner.fit_onecycle(2e-5, 1) 



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 5398s 1s/step - loss: 0.3543 - accuracy: 0.8430 - val_loss: 0.2541 - val_accuracy: 0.8931


Observations:
1. Using Ktrain was 100 times easier than with manual implementation.
2. With less amount of data cleaning i managed to achieve Decent validation accuracy.
3. In my opinion BERT is a data hungry algorithm. Intensive data cleaning will hurt the model performance as it reduces the data accordingly (eg. lemmatization, removal of stopwords etc.).
4. A better approach would be to try out deep learning algorithms (LSTMs, BERT etc.) with and without intensive data cleaning and compare the results. 